
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img
    src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png"
    alt="Databricks Learning"
  >
</div>


## 1.10 Lab - Dashboard and Visualization Lab Activity

Databricks AI/BI Dashboards help you quickly transform data into shareable insights. 

In this lesson, you will create a dashboard and make it useful by adding text, data, and interactive visualizations.

This lesson uses the following resources:
- Tables:
  - Catalog: dbacademy
  - Schema: {dynamically named, personal to you}
  - Tables:
    - au_orders
    - au_opportunities
    - au_customers

These tables contain simulated business-to-business order and opportunity data for an imaginary company's Australian sales operation. The **au_orders** table contains information from 2021 through the middle of November 2024, and the **au_opportunities** table contains the opportunities present in our company's Customer Relationship Management (CRM) software as of that time. Both of those tables identify the relevant customer with a unique key that points into the **au_customers** table. You'll use the data in these tables to prepare your dashboard.


### REQUIRED: Course Setup and Data Discovery
The first thing you'll need to complete any data analytics task is the appropriate data for the request. To find the right data, you'll need to use the Catalog Explorer or the Databricks Search to locate the data for this project.  In this particular lab exercise, a setup script will copy tables from a Databricks Marketplace share into a schema you control. To get started, click the small triangle Run button in the top left of the cell below.



---

In [0]:
%run "../Includes/setup/au_setup"

This script clones a few small data tables from a Databricks Marketplace share into your personal schema. At the end it prints out the name of your catalog and your schema. The schema will have a randomly generated name. Make a note of this value. In these instructions, we'll call this simply "your schema."

To use the Catalog Explorer, follow the steps below.

1. Select **Catalog** from the sidebar navigation.
1. In the catalog selector, locate the catalog titled: **dbacademy**. You can also use the search at the top to narrow down the available options.
1. Expand your schema. You should see three tables in this schema.
    - au_orders
    - au_opportunities
    - au_customers

### A: Create a new Dashboard
The first step in creating a new dashboard in Databricks is to decide upon the business purpose of the dashboard and its audience. For this dashboard, the business purpose will be to provide information on the performance of our Australia sales operation. The intended audience is any employee of our business who supports and evaluates our sellers.

**📌 NOTE:** Many of the following instructions take place in a non-notebook area of the platform. It is recommended to open either the notebooks in a separate tab or window to reference the instructions alongside the area where they are performed.

In the Databricks user interface, create a new, blank dashboard. 
* Navigate to **Dashboards** in the side navigation pane.
* Select **Create dashboard**. 
* At the top left of the dashboard definition panel, click on the placeholder dashboard name ("New Dashboard" followed by a timestamp) and change it to **Australia Sales**.



### B: Examine and Augment the Main Dataset

With a completely new Dashboard, you need to associate the dashboard with data before you can begin designing the information display itself. At the top of the dashboard screen, you have two tabs, **Data** and **Untitled page**. 

- **Data:** The **Data** tab allows you to define datasets that you will use in the dashboard. Datasets are bundled with dashboards when sharing, importing, or exporting them using the UI or API.

- **Untitled page:** This is the dashboard canvas that allows users to create visualizations and construct their dashboards. Each item on the canvas is called a widget. Widgets have three types: visualizations, text boxes, and filters.

You start off on the **Untitled page** tab so that you can see the helpful overview that's provided. Select the **Data** tab to get started. 

There are three small icons on the left side of the **Data** tab's screen area: **Datasets**, **Catalog**, and **Assistant**. 
* **Datasets** (an icon depicting a list) will present you with a list of all the datasets used for the dashboard. This is where we start; it's empty now.
* **Catalog** (the standard Databricks Catalog icon, three tiny shapes) allows you to navigate the available catalogs, schemas, and tables accessible to the workspace and select tables or materialized views to use as datasets.
* **Assistant** (the standard Databricks Assistant icon, a tiny four-pointed star) provides you with a AI-powered interface for asking queries in natural language to the platform to discover objects or gain insights or assistance on query writing. 

  **📌 NOTE**: 

  - To make sure that subsequent instructions work literally, be sure that the field containing the customer's state is called **customer_state**. Of course you can always call it something else, but you'll need to adapt subsequent lab instructions. 

  - Throughout these instructions, replace **`YOURSCHEMA`** with the name of your schema. (You can find your schema name in the setup run cell executed earlier in this notebook.)

1. With the **Datasets list** icon selected, click the **+ Create from SQL** button. Right-click the newly created query in the **Datasets** list and select **Rename**, or use the kebab menu, to rename the query as **orderhistory**.  


2. Let's borrow the SQL editor panel to look at the data in the **au_orders** and **au_customers** table. First, let's see what's in the **au_orders** table. Paste in this query:

```
SELECT * from dbacademy.YOURSCHEMA.au_orders LIMIT 10;
```

  Then click the **Run** button. 

  We see an order ID field, which is an integer; and next we see two ID fields consisting of incomprehensible strings, one field for the customer and one for the product. Next we see the date on which the order was placed, the quantity ordered, the total price of the order, and the name of the sales rep who managed the deal. It would be great if we could get more information about these customers and products, wouldn't it? For example, their names, and for the customers, where they are located. 

3. Fortunately, our schema contains a **au_customers** table. Use your mouse to clear the previous query from the SQL editor panel and paste this one in instead:

```
SELECT * from dbacademy.YOURSCHEMA.au_customers LIMIT 10;
```

  Then click the **Run** button. 

  We see an customer ID field, containing those same meaningless strings, but here for each we see the customer's name, their city, and their state (using standard Australian state abbreviations, which are either two or three letters). We should be able to use this table to look up information about our customers for our visualizations. For example, now we can report on all the customers who are in particular states.

4. But what about the products? There does not seem to be a similar table that explicates each product ID in the same way that **au_customers** does for the customer IDs. In the real world, it is very common to need to enhance and augment data to get good visualizations, and that is exactly what we are going to do. Fortunately, in this example company, there are only 9 products, so we can quickly create a small table with more information about each.  _In this screen_, not in the Databricks user interface, click the small "Play" icon (the rightward-pointing triangle) at the top of the code block immediately below these words.

In [0]:
-- Remember, don't paste this cell into the Dashboards SQL editor panel.
-- Just click the small "Run" triangle in this box's top left corner
DROP TABLE IF EXISTS dbacademy.YOURSCHEMA.au_products;

CREATE TABLE dbacademy.YOURSCHEMA.au_products (productid string, productname string, listprice float);

INSERT INTO
  dbacademy.YOURSCHEMA.au_products (productid, productname, listprice)
VALUES
  ('JQ9322', 'TableSaw', 24.99),
  ('A802a', 'Mill', 10.17),
  ('EE05x', 'Press', 363.02),
  ('R9S10', 'Lathe', 43.83),
  ('BC111d', 'Drill', 63.28),
  ('W8931', 'Planer', 45.22),
  ('Q28R3', 'Grinder', 176.33),
  ('MDb304', 'CAD System', 502.80),
  ('OR307p', 'Kazoo', 0.99);

5. Now return to the Databricks user interface window. Return to the SQL editor panel in the Dashboards dataset definition screen. Use your mouse to clear the previous query from the SQL editor panel and paste this one in instead:

```
SELECT * from dbacademy.YOURSCHEMA.au_products;
```
Then click the **Run** button. 

Now we have a table called **au_products** with human-friendly information about each product.

### C: Define the Main Dataset

Now we have enhanced our schema with useful information for our dashboard's audience, who cannot be expected to recognize those arbitrary product IDs. In the real world, you would be more likely to enhance your data by _uploading_ the data for a new table, such as a Comma-Separated Value file ("CSV file"). These CSV files are commonly created through exporting spreadsheets. Instead, we created a table using Databricks SQL. Nevertheless, we are finally ready to create a table with everything we need to help our dashboard's end users.

1. Be sure that you are in the Databricks user interface window, and in its Dashboards dataset definition screen. The **orderhistory** dataset should be selected. Use your mouse to clear any previous query from the SQL editor panel. 
1. Now click the Assistant icon at left. 
1. In the Assistant chat box, describe the SQL query you need. 
 Specifically: 

    You want a query that returns all the fields in **dbacademy.YOURSCHEMA.au_orders**, plus the customername, city, and state fields from **dbacademy.YOURSCHEMA.au_customers**, joining those two tables on the **customerid** field. You also want to pull in the **productname** field from **dbacademy.YOURSCHEMA.au_products**, joining those two tables on the **product** field. The customer's state should be called **customer_state**.

     Feel free to use the above query description or rewrite it in your own words.

 ---
**TIPS FOR SUCCESS**

* You don't have to get the Databricks Assistant to produce the perfect SQL query. Use it to get  close to the goal, and then touch up the final query as needed. 

* In the context of defining a dashboard, always name tables with their full three-part form: _catalog_._schema_._table_. 

* Sometimes Databricks Assistant guesses wrong about the exact name of the field that binds two tables together. You can tell it explicitly what field to join on, as we did in our query description above. Or your data engineers can help Databricks Assistant in a more permanent way by defining the underlying tables to have a [constraint clause](https://docs.databricks.com/en/sql/language-manual/sql-ref-syntax-ddl-create-table-constraint.html). 

---

 Feel free to use the above query description or rewrite it in your own words. 
1. Copy the SQL query that Assistant suggests and paste it into the SQL editor panel in the center of the screen (over the words "Start typing...").
1. Run your query and see if it works! You may need to adjust it. If there is an error, Assistant will display a "Diagnose error" button to inspect your SQL and offer a corrected query. Copy the new query back into the SQL editor panel, overwriting the previous attempt, and try to run it again. You might need to use the "Diagnose error" button more than once.

    If you get stuck, a sample query you can use is at the end of this lab exercise. 

1. If you like, you can close the Assistant window by clicking the X at top right. But you can always return to it, with full history, by clicking the Assistant icon.



### D: Add a Dataset for the 2024 Sales Goal

1. Still in the **Data** tab, select the **+ Create from SQL** option. 
2. Enter the following query into the query editing space:
```
  SELECT 
      sum(orderamt) AS Total_Sales, 390000000 AS Sales_Goal 
  FROM 
      dbacademy.YOURSCHEMA.au_orders
  WHERE 
      orderdate >= '2024-01-01' AND orderdate <= '2024-12-31' ;

```
3. Click **Run** to execute the query. 
4. Right-click the newly created query in the **Datasets** list and select **Rename**, or use the kebab menu, to rename the dataset it constitutes as **SumTotalSales**.

Now your dashboard has a total of two datasets.

### E: Add Visualizations
---
#### Adding a Text Box

Let's add a name and a space for a text description of the dashboard to the canvas. When adding a new widget to the canvas, other widgets automatically move to accommodate your placement. You can use your mouse to move and resize widgets. To delete a widget, right-click it and choose **Delete** from the resulting context menu. 

Complete the following steps to add a text box to the dashboard:

1. Click on the **Untitled page** tab at top right to switch to the canvas view.
1. At the bottom of the screen you have a palette for adding various widgets. Click the icon in the palette for adding a text box and drag the resulting widget to the top of your canvas. The text-box icon looks like a T in a small rectangle.
1. Click into the just-placed widget and type: `# Australia Sales`

    **📌 Note:** Text boxes use [Markdown](https://daringfireball.net/projects/markdown/syntax) syntax. The single `#` character in the included texts indicates that <b>Australia Sales</b> is to be styled as a level-1 heading. 

1. On the next line, type a brief explanation of what this dashboard is for, such as "This dashboard helps you support and evaluate our Australia sales operation". Append an invitation for users to email you with feedback using Markdown's embedded-link syntax:

   `[Send me feedback!](mailto:your@email.address)`


1. Now open up a new line above the Australia Sales header. Here you will add a company logo. First type the Markdown syntax for an embedded image:

    `![company logo]()` 

    (Whatever is between the square brackets will be treated as alt text.)

    Now get an image URL into your browser's clipboard. You can right-click on the image below and capture its link address to use it, or you can use another logo, such as your own company's. Just make sure that the image is small. (Pro-tip: open the image in a new browser tab to confirm that it is small before proceeding.)

    ![Description](../Includes/images/newlogo4_sm.png)

    Paste the URL between the parentheses (not the brackets!). 


1. Drag the right edge of the text widget inward so that it is about 1/2 the width of the dashboard. You are making room for a widget to sit to the right of the title. Adjust the bottom edge as needed so that all the contents of the text box are visible.


---
#### Adding a Counter

The first visualization we'll be adding to the dashboard is a counter visualization to display the current sales against a sales goal of $390 million.


1. Return to the **Untitled page** tab if you navigated away.  
1. Click the icon in the palette for adding a visualization, which is a tiny thumbnail of a line chart.  
1. Move your cursor to anywhere on the screen and click to add the visualization to the canvas.  
1. In the configuration panel on the right, make the following selections for the settings:
    - **Dataset:** **SumTotalSales**
    - **Visualization:** Counter
    - **Title:** Checked
      - Click on **Widget Title** on the visualization.
      - Change it by typing over it to **Sales Goal**.
    - **Value:** Total_Sales
    - **Comparison:** Sales_Goal

1. Click on **Total_Sales** in the **Value** area of the configuration panel and select **Format** from the resulting dropdown. Make the following adjustments:
    - Change **Auto** to **Custom**.
    - Set **Type** to Currency ($)
    - Change the Currency to Australian Dollars ($)

1. In the Style section, click the **+** next to **Conditional Style**. Configure it with the following settings:
    - If Value <= Target
    - Then (Color: Red)

Adjust the width of the counter visualization so that it is 1/2 the width of the dashboard, then drag the edges or click-hold while hovering over the visualization box to put it to the right of the title box. Adjust the height of the counter visualization so that it is 1/2 the height of the adjacent text box. 

Try this experiment: change the currency of the sales total to British Pounds (£). Does the value displayed change, or just the currency symbol? Change the currency back to Australian Dollars ($) before proceeding.

---
#### Adding a combo chart
Next, let's add another visualization, this time a Combo Chart containing information about total sale price and count of sales over a three months span.

1. Return to the **Untitled page** tab if you navigated away.  
1. Click the icon in the palette for adding a visualization.
1. Land the new widget on the left side of the canvas underneath the text box. Click in the new widget to give it focus. 
1. In the **Configuration Panel** on the right, make the following selections for the settings:
    - **Dataset**: **orderhistory**
    - **Title:** Checked
      - Click on **Widget Title** on the visualization.
      - Change it by typing over it to **Order History**.
    - **Visualization**: Combo
    - **X axis**: Click the **+** and select **orderdate**.
      Notice that the dialogue guesses that we want to view the data on a monthly basis by replacing **orderdate** with **MONTHLY(orderdate)**.

        Click on **MONTHLY(orderdate)** and fill in **Order date** for the **Display name**.
    - In the **Y axis** section:
      - **Bar**: Click the **+** and select **orderamt**.
      Notice that the dialogue guesses that we want to sum the order amounts by replacing **orderamt** with **SUM(orderamt)**. It also automatically knows that these sums should be on a monthly basis, because that is the grouping selected for the X axis.

        Click on **SUM(orderamt)** and fill in **Total Sales Value** for the **Display name**.
      - **Line**: Click the **+** and select **COUNT(*)** (which is offered at the end of the list of fields; this is SQL syntax for "count the number of records")

          Click on **COUNT(*)** and fill in **Count of Sales Orders** for the **Display name**.

1. So far this doesn't look very useful, does it? The line for **Count of Sales Orders** hugs the X axis because the Y axis covers values in the millions (of dollars). Let's fix this. Select the kebab menu icon next to **Y axis** and click **Enable dual axis**. Enabling dual axis allows us to see changes in the data on two different scales. 

  You can also adjust the colors for the series by selecting the color blocks next to series names.

---
#### Adding a bar chart using Databricks Assistant

When drafting a dashboard, you can provide a natural language prompt to the Databricks Assistant and it autogenerates a chart based on your request. The Databricks Assistant can help you build charts based on any dataset defined in your dashboard's data tab. Let's try it out.

Complete the following steps:

1. Return to the **Untitled page** tab if you navigated away.  
1. Click the icon in the palette for adding a visualization.
1. Land the new widget on the left side of the canvas underneath the text box. Click in the new widget to give it focus.
1. In the "Ask the Assistant to create a chart..." box at the top of your new widget, enter the following prompt:  

    _Create a bar chart from the orderhistory dataset with productname on the x-axis and orderamt on the y-axis_

1. Click the paper-airplane icon to generate a response. It may take a moment for the Assistant to provide a visualization.  
1. You will get a bar chart that matches the provided description. Click <b>Accept</b> to confirm the visualization meets your needs.  

    If the visualization does not match your description or match the kind of visualization you wanted to create, you can reject or regenerate the response. You can also adjust the configuration of the chart once accepted. 

1. Change the color of the chart to something other than the default. Use the configuration panel's **Color** section.
1. Check the **Title** box in the configuration panel. Then click on the placeholder widget title on the visualization. Change it by typing over it to **Average Order Amount per Product**.


---
#### Adding a pie chart

Let's show how our revenue is distributed among the Australian states by adding a pie chart. To make this pie chart, complete the following steps:

1. Return to the **Untitled page** tab if you navigated away.  
1. Click the icon in the palette for adding a visualization.
1. Land the new widget on the left side of the canvas underneath the text box. Click in the new widget to give it focus.  
1. In the **Configuration Panel** on the right, make the following selections for the settings: 

    - **Dataset**: **orderhistory**
    - **Title:** Checked
      - Click on the placeholder widget title on the visualization.
      - Change it by typing over it to **Revenue Share by State**.
    - **Visualization**: Pie
    - **Angle**: **orderdate**

      Click on **orderdate** and fill in **Revenue** for the display name.
    - **Color**: Click the **+** and choose **customer_state**.
      Click on **customer_state** and fill in **State** for the display name.




---
#### Adding a heatmap

Let's examine how our order totals vary across Australia both by the state and also by the sales rep who executed the order.

1. Return to the **Untitled page** tab if you navigated away.  
1. Click the icon in the palette for adding a visualization.
1. Land the new widget on the left side of the canvas underneath the text box. Click in the new widget to give it focus.  
1. In the **Configuration Panel** on the right, make the following selections for the settings: 

    - **Dataset**: **orderhistory**
    - **Title:** Checked
      - Click on the placeholder widget title on the visualization.
      - Change it by typing over it to **Distribution of Order Quantity across States and Sales Reps**.
    - **Visualization**: Heatmap
    - **X axis**: **salesrep**

      Click on **salesrep** and fill in **Rep** for the display name.
    - **Y axis**: Click the **+** and choose **customer_state**.
      Click on **customer_state** and fill in **State** for the display name.
    - **Color**: Click the **+** and choose **quantity**.
      Click on **SUM(quantity)** and fill in **Qty** for the display name.

      Click on the little color swatch and choose a more fun color ramp than the default.

Which sales rep probably has Sydney in their territory?




---
#### Cross-filtering, and adding filter widgets

Databricks AI/BI Dashboards offer user interactivity even if you, the dashboard author, don't do any extra work. To see this, hover over the Order History bar chart's bars. A tooltip pops up for each sales-value bar giving its exact value, as well as the sales-count line's value for that month.

That's not all. Try clicking on one of the boxes in the heatmap. Notice that all the other visualizations adjust to focus on that sales rep and that state. To undo that adjustment, click again on the same box you clicked on. Now try clicking on the segments of the pie chart. This feature, called cross-filtering, lets your users get more information out of your dashboards with no extra work on your part and no need for them to ask you for help. (Again, to undo the adjustment, click once more on the pie-chart segment you clicked on.)

But what if a user wants to refine an entire visualization in a more complex or precise way? Maybe they want to hone in, say, a time-range or a product or a sales rep of interest. Filters are widgets that allow dashboard viewers to do just that. All you have to do is to build the appropriate widget into your dashboard, and AI/BI Dashboards does the rest.

Local filters automatically apply to all the visualizations on the same page that share their dataset. That means that the more distinct datasets your dashboard is built on, the more filter widgets you might have to offer, and the more you'll have to help users understand which filters work with which visualizations. It is wise to be economical with the number of datasets in a dashboard.

To add a filter on orders across a user's preferred date range to the dashboard, complete the following steps:

1. Return to the **Untitled Page** tab if you navigated away.  
1. Click the icon in the palette for adding a filter, which looks like a tiny funnel.
1. Land the new widget wherever it is convenient for now. Click in the new widget to give it focus.
1. When the filter widget is selected, the filter configuration panel appears on the right side of the screen.

1. Apply the following settings:  
  - **Filter**: Date range picker
  - **Fields**: Expand the **orderhistory** dataset and choose **orderdate**
6. Use the checkboxes to turn on **Title**.  

1. Click the placeholder title on the new filter widget and change it to **Order Date**  

Let's add a global filter. To add a global filter for products to the dashboard, complete the following steps:

8. Return to the **Untitled page** tab if you navigated away.  
1. Click the filter icon to the left of **Untitled page** at the top of the canvas.
1. You'll notice a new panel opens on the left for filters. Click the **+** to add a global filter. Click in the new filter to give it focus if not already selected.
1. When the filter is selected, the filter configuration panel appears on the right side of the screen.

1. Apply the following settings:  
  - **Filter**: Multiple values
  - **Fields**: Expand the **orderhistory** dataset and choose **productid**
13. Use the checkboxes to turn on **Title**.  
1. Click the placeholder title and change it to **Product**  

1. Experiment with your filter. Notice that both the graph-style visualizations you have added so far respond to the date ranges you supply, while the **Sales Goal** counter widget does not. Why not? Did we make a good decision when we used a separate dataset to define **Sales Goal**? (Hint: our organization has established a sales goal that goes across the entire year and across all products.)
1. To clear the date filter and return to a display of all the data, click the tiny calendar icon at the right edge of the filter box and choose **All** from the resulting pulldown.
1. Move the local filter widget you created into the empty space underneath the sales-goal counter widget. Now it is at the top of the dashboard, a very visually prominent location that will remind users when a filter is active. You can close the global filters panel by clicking the **X**.




---
The image below is supplied as an example of how your dashboard's first page could appear once you've finished adding visualizations and customizing the colors and features of the dashboard. The lab activity continues below the image.

![Description](../Includes/images/australia_sales_dashboard_lab_solution_new.png)

### F: Adding another page to the dashboard

Sometimes it is helpful to let people browse the underlying data. Let's give that opportunity, but put it on a separate page, so as not to clutter up the main page. 

1. Return to the **Untitled page** tab if you navigated away.  

1. Because we are now going to have two pages, let's rename the first one to something helpful. Click the kebab menu next to **Untitled page**, choose **Rename**, and specify a new name of **Visualizations**.

1. Now click the `+` icon to the right of **Visualizations** to add a new page. Click the kebab menu next to the newly created **Untitled page**, choose **Rename**, and specify a new name of **Browse data**.

---
#### Adding a table widget to the dashboard's second page

Now that we have a separate page, we can let people browse the underlying data if they wish without cluttering our handsome graphs.

1. Return to the **Browse data** page tab if you navigated away.  

1. If you are not already viewing the page called **Browse data** click on its tab at top left to toggle to it.

1. Select **Add a visualization** from the menu at the bottom of the canvas and click on the canvas to add the visualization. 

1. In the **Configuration Panel** on the right, make the following selections for the settings: 

    - **Dataset**: **orderhistory**
    - **Visualization**: Table
    - **Columns**: 
      - Click the box for **Show/hide all** to add all the fields to the table.
      - Click the eye icon next to the **orderid**, **customerid**, and **productid** fields to hide them.

1. Grab the right edge of the widget and expand it to the full width of the page.

Notice that, just as with a spreadsheet, your users can sort all the data by clicking on the header. Clicking once selects ascending order and clicking again selects descending order.

Does the date selection widget on the front page of the dashboard affect this table? What about the global filter for product?


### G: Securely share your dashboard

When your dashboard is complete, you can publish it and share it with others. 

Published dashboards can be shared with other users in your workspace and with users registered at the account level. That means that users registered to your Databricks account, even if they have not been assigned workspace access or compute resources, can be given access to your dashboards.

When you publish a dashboard, the default setting is **Share data permission**. Share data permission in your published dashboard allows dashboard viewers to use your data permissions to access the data and power the queries that support it. This enables a shared cache, which can improve performance. If you choose **Individual data permission**, each viewer of the published dashboard uses their own data permissions to run queries. This can lead to more frequent refresh operations. If a viewer does not have access to the underlying data or the default SQL warehouse that powers the dashboard, _visualizations will not render._  

You can also notify viewers by sending emails to individuals or small groups when the dashboard is published.

To publish your dashboard, complete the following steps:

1. Click **Publish** in the upper-right corner of your dashboard. Read the setting and notes in the **Publish** dialog.

1. In the **Publish** dialog, you’ll see two data permission options. Make sure to select **Share data permission**.
1. Optionally, you can notify viewers via email and describe your changes in the text field.
2. Click **Publish** in the lower-right corner of the dialog. The **Sharing** dialog should open afterward. If it does not open, you can select **Share** next to **Publish** at the top of the dashboard.
    - You can use the text field to search for individual users, or share the dashboard with a preconfigured group, like **Admins** or **All workspace users**. From this window, you can grant leveled privileges like **Can Manage** or **Can Edit**. See <a href="https://docs.databricks.com/en/security/auth-authz/access-control/index.html#lakeview" target="_blank">Dashboard ACLs</a> for details on permissions.
    - The bottom of the **Sharing** dialog controls view access. Use this setting to easily share with all account users.
3. Under **Sharing settings**, choose **Anyone in my account can view** from the drop-down. Then, close the **Sharing** dialog.
4. Use the drop-down near the top of the dashboard to switch between **Draft** and **Published** versions of your dashboard.

**📌 Note:** When you edit your draft dashboard, viewers of the published dashboards do not see your changes until you republish. The published dashboard includes visualizations that are built on queries that can be refreshed as new data arrives. When dashboards are refreshed, they updated with new data automatically; the dashboards do not need to be republished.

### H: Establish a refresh schedule

A refresh schedule helps you optimize the performance of  dashboards built on datasets that grow and change. You can schedule a regular refresh of the data in your dashboard, thus ensuring that Databricks' cache of data is up-to-date. The refresh schedule dialogue is also where you can manage users' subscriptions to your dashboard. Subscribed users get an email at every refresh with a snapshot image of the first page of the dashboard, as well as a link back to the live dashboard in Databricks.

1. Navigate to the published view of your dashboard if you are not already there. The **Draft**/**Published** pulldown at top should show a green circle.
1. Click on the **Schedule** button at top right. If applicable, select **+ Schedule**.

1. Use the user interface to establish a daily refresh schedule. Choose a time that is in a few minutes.
1. Click to **Create** the schedule.
1. Click the **Schedule (1)** button again and use the **Subscribe** button to subscribe to the refresh schedule.
1. Using the kebab menu icon to the right of the **Subscribe** button, select **Edit**. From here you can switch the view to the **Subscribers** tab and view all users who've subscribed to that particular refresh schedule.
1. Select **Cancel** to leave the **Edit schedule** dialogue.

  In a production Databricks instance you would use this user interface to manage the subscribers to your dashboard. As a security measure, you cannot subscribe arbitrary email addresses to your dashboard. Only users with a Databricks account in this Databricks workspace can subscribe.

  1. Click the **Schedule (1)** button again. Using the kebab menu icon to the right of the **Subscribe** button to **Delete** the schedule.

---
### Appendix: Sample SQL Query

Here is a query you can use to define the dashboard's main **orderhistory** dataset.

```
SELECT
    o.*,
    c.customername AS customername,
    c.state AS customer_state,
    p.productname AS productname
FROM 
    dbacademy.YOURSCHEMA.au_orders AS o
JOIN 
    dbacademy.YOURSCHEMA.au_customers AS c
    ON o.customerid = c.customerid
JOIN 
    dbacademy.YOURSCHEMA.au_products AS p
    ON o.productid = p.productid ;
```




---
### Challenge activity: Opportunities page

If you have time, enhance your dashboard by adding a third page and presenting helpful visualizations of the opportunities table. To do so, you'll need to define a new dataset that joins the **opportunities** table to the **au_customers** table and then explore the data in the table. What fields are present? What visualizations might be interesting? Consider visualizations that compare the opportunities by sales phase and by sales rep.

&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="_blank">Apache Software Foundation</a>.<br/><br/><a href="https://databricks.com/privacy-policy" target="_blank">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use" target="_blank">Terms of Use</a> | <a href="https://help.databricks.com/" target="_blank">Support</a>